In [14]:
from transformers import AutoModel, AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification,TrainingArguments, Trainer
from datasets import load_dataset
import os
import torch

In [2]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
model = AutoModelForSequenceClassification.from_pretrained("./phoBert-sa/checkpoint-5000")

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [5]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [78]:
test_dataset = load_dataset('csv', data_files={'test' : '../../data/processed/test.csv'})

Using custom data configuration default-27af0524cb2f98b1
Reusing dataset csv (/home/khuongnd6/.cache/huggingface/datasets/csv/default-27af0524cb2f98b1/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

In [79]:
test_dataset

DatasetDict({
    test: Dataset({
        features: ['Unnamed: 0', 'id', 'free_text'],
        num_rows: 5086
    })
})

In [80]:
show_random_elements(test_dataset["test"])

,Unnamed: 0,id,free_text
0,1623,test_nuiaarqahm,ngô quang huy nét đẹp sinh_viên cofer ảnh dự thi của bạn mai thị ái như lớp tcdnc giới_thiệu bản_thân mình là cô gái luôn mang tư_tưởng sống và cố_gắng hết_mình để tận_hưởng cuộc_sống một_cách trọn_vẹn nhất có_thể mình đặc_biệt thích đi du_lịch khám_phá những nơi mà mình chưa từng đến bên cạnh đó mình còn hứng_thú với việc ca_hát múa dân_gian và biên_đạo văn_nghệ cho các cuộc thi nhỏ châm_ngôn sống của mình là cuộc_sống không bao_giờ là bế_tắc thực_sự hay có khái_niệm mất tất_cả một_khi bạn còn có niềm tin link facebook
1,3286,test_arelbvwrut,vương trúc hồng shared memory with nguyễn_thị lợi and như vương
2,5063,test_lmbkeudfbe,hoàng long trúng nhớ chia
3,2429,test_twonfviwrq,đắt vl ấy
4,750,test_yclqtkzfmq,ngân cherry dien quan entertainment giải_trí đứa nào lớn miệng hơn đứa đó thắng dienquan entertainment
5,1383,test_nkfyosyfra,bông nói em nhăn mặt giống bố nè
6,2072,test_ebztnvnfmm,xem ib nhé
7,3728,test_jpmscvvnze,mấy chế ai sắp về việt_nam thích ăn gi an ít lại đặc_biệt là mấy loại thịt đang có dịch_bệnh em về lần ăn quá_trời giờ cái miệng bi ngứa và sưng tấy bác si bảo đã di ứng với thức_ăn và bị tiêm mũi phòng_chống dịch_bệnh và mũi dị_ứng ui mẹ ơi nhức môi rồi vô đau luôn cái mông trân_châu pong is feeling tired
8,3648,test_tnhephnnun,đổi các loại pass vẫn phải quay về pass cũ nhọc vl
9,1685,test_pdwdekwtft,trọn bộ mền trần bông ga vỏ gối giá có sẵn vải poly mềm mát hình kết ko xù ko phai ktra hàng thoải_mái nhé photos from truc nguyen post


In [85]:
mapped_result = {0 : 'CLEAN', 1 : 'OFFENSIVE_BUT_NOT_HATE', 2 : 'HATE'}

def map_to_result(batch):
    if batch['free_text'] == None:
        batch['label_id'] = torch.tensor([0],device="cuda")
        return batch
    else:
        model.to("cuda")
        input_values = tokenizer(batch['free_text'],return_tensors="pt").to("cuda")
        with torch.no_grad():
            logits = model(**input_values).logits
        batch['label_id'] = torch.argmax(logits, dim=-1)
        return batch 

In [86]:
test_dataset = test_dataset.map(map_to_result)

  0%|          | 0/5086 [00:00<?, ?ex/s]

In [92]:
show_random_elements(test_dataset["test"])

,Unnamed: 0,id,free_text,label_id
0,4677,test_bksqlqwbyp,phạm hoa chuẩn,[0]
1,1764,test_aewqqbxjvg,bữa khách nào hỏi mà em quên mất rồi hnay về lại nhé mndụng cụ phòng bắt_cóc chống thất_lạc cdễ dàng kiểm_soát bé nơi công_cộng hoặc đông người chợ siêu_thị công_viên trinh ngoc vuong added new photos photos from trinh ngoc vuong post,[0]
2,3548,test_uirxserqjk,có ai đang dùng dung_dịch vệ_sinh phụ_nữ mà dùng nhiều lại sợ bị viêm mà dùng thì đc ib chỉ cho nè,[0]
3,2663,test_erlcgxvsbf,chỗ đẹp đồ_ăn ngon những chiếc ảnh xinh xắnkhông cãi nhau nữa là hoàn_hảo sang lê_added new photos in hội_an photos from sang lê_post,[0]
4,763,test_leftnyygbj,cap ngầu vl,[0]
5,5067,test_iisuwmpvrn,tháng cho lạnh,[0]
6,1905,test_xnugxpavcd,share lại cho nhớ cái bản_mặt phạm hương phạm chúc_mừng sinh_nhật lợn con nhà tôi ngọc anh có thần_thái từ bé cơ,[0]
7,4866,test_gqpwartppk,thanh_thuỷ cute vl,[0]
8,1414,test_lanunohtrm,con là coca đẹp dai mỗi_tội ngáo ngơ photos from nguyễn_đức_cường post,[0]
9,310,test_tbnsysvswp,nhác vlone,[0]


In [102]:
def map_tensor_to_value(batch):
    batch['label_id'] = batch['label_id'][0]
    return batch

In [103]:
test_dataset = test_dataset.map(map_tensor_to_value)

  0%|          | 0/5086 [00:00<?, ?ex/s]

In [107]:
show_random_elements(test_dataset["test"])

,Unnamed: 0,id,free_text,label_id
0,2163,test_spzhgfbicy,ngân trần diệu hoàng đâyyy bảo khôi ngân iphone thì có ai đó thì chưaaa,0
1,100,test_bwgpnwqgeg,thiên bình shared post pho chỉ có con gái mới mang hạnh_phúc lại cho nhau yeahnewsphim tình_yêu không có lỗi lỗi bạn thânlink youtube,0
2,1689,test_hryxmdzbsq,lợi em zia test,0
3,4179,test_ihcphpgewk,tao cũng nghĩ vậi,0
4,387,test_jedrleohsu,giàu giàu giàu giàu,0
5,944,test_oxxcxqidbl,nhiều áo mới lắm ahcác nàng ghé xem nhé áo_cưới như ýđường trần phú khóm tt mỹ an tháp_mười dãy dũng đẹp đường chiều chơ tháp_mười như võ is with võ ngọc nguyên and others photos from như võ post,0
6,1324,test_lzberhnvat,ngồi soạn từ mà quay đi quay lại đã đêm rồi các bác cứ bật máy chế_độ thường còn nhận hàng nhé chứ để máy_bay là em chịu thôi hihi không ngờ saffron được các chị_em tin_yêu đến thế hàngđimỗingày photos from tình tính tang post,0
7,3835,test_pfazsaalqw,làng đo đo nơi đó có chị rùavà dòng sông kiếp bạc hay vô_cùng,0
8,3019,test_gydttacegu,quyết_định cung thể quyết_định lai,0
9,514,test_jwnoqtgrog,về đc mã quần lửng chất rất đẹp nha cedáng baggy tmai chất co_dãn mạnhsize photos from lê_thị minh thư post,0


In [116]:
test_dataset["test"][4902]

{'Unnamed: 0': 4902,
 'id': 'test_pgwxdgqmqy',
 'free_text': 'mát vl đú cc',
 'label_id': 0}

In [108]:
result = test_dataset['test'].remove_columns(['Unnamed: 0','free_text'])

In [109]:
result

Dataset({
    features: ['id', 'label_id'],
    num_rows: 5086
})

In [111]:
result.to_csv('../../data/submission/submission.csv',index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

91560

In [117]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN